<a href="https://colab.research.google.com/github/roddrick369/mirror-bot/blob/main/indexer_bot_%5Bupd8%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tgcrypto
!pip install IPython nest_asyncio
!pip install telethon
!rm -rf /content/temp_downloads/*.part  # Limpar arquivos temporários antigos
!mkdir -p /content/temp_downloads  # Garantir estrutura de diretórios

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os
import re
import asyncio
import tempfile
import mimetypes
import json
import hashlib
from threading import Lock
import nest_asyncio
from collections import defaultdict
import time
from datetime import datetime
from telethon.tl.functions.messages import ExportChatInviteRequest, DeleteChatUserRequest
from telethon.tl.types import ChatInviteExported, InputUserSelf
from telethon import TelegramClient, events, errors
from telethon.tl.functions.channels import JoinChannelRequest, CreateChannelRequest, CreateForumTopicRequest, LeaveChannelRequest
from telethon.tl.functions.messages import CreateChatRequest, ImportChatInviteRequest, DeleteChatUserRequest
from telethon.tl.types import (
    MessageService,
    MessageMediaPhoto,
    MessageMediaDocument,
    MessageMediaWebPage,
    DocumentAttributeFilename,
    DocumentAttributeVideo,
    Channel,
    Chat,
    PeerChannel,
    PeerChat,
    MessageReplyHeader
)
from telethon.errors import (
    FloodWaitError,
    ForbiddenError,
    ChatWriteForbiddenError,
    RPCError,
    ChatForwardsRestrictedError
)

mimetypes.add_type('video/x-matroska', '.mkv')  # Para vídeos MKV
mimetypes.add_type('image/webp', '.webp')  # Para stickers animados

# Configuração do Colab
# nest_asyncio.apply()

# Montar o Google Drive (execute apenas uma vez por sessão)
#if not os.path.exists('/content/drive/MyDrive'):
#    from google.colab import drive
#    drive.mount('/content/drive')

# Configurações de persistência
DRIVE_PATH = '/content/drive/MyDrive/IndexerBot/'  # Crie esta pasta manualmente no seu Drive
SESSION_NAME = os.path.join(DRIVE_PATH, 'session')  # Nome fixo para reutilizar a sessão
PERSISTENCE_FILE = os.path.join(DRIVE_PATH, 'session_state.json')
LINKS_DB = os.path.join(DRIVE_PATH, 'private_links.db')
PROCESSED_MSGS_FILE = os.path.join(DRIVE_PATH, 'processed_messages.json')
LAST_CHECKPOINT_FILE = os.path.join(DRIVE_PATH, 'last_checkpoint.json')  # Novo arquivo para checkpoints

# Garantir que o diretório existe
if not os.path.exists(DRIVE_PATH):
    os.makedirs(DRIVE_PATH)

QUEUE = asyncio.Queue()
WORKERS = 1
MAX_CONCURRENT_PER_USER = 1
BATCH_SIZE = 50
FORWARD_DELAY = 180

class UserState:
    def __init__(self):
        self.active_operations = 0

user_states = defaultdict(UserState)
mirror_channels = {}
forum_topics = defaultdict(lambda: defaultdict(dict))
lock = Lock()

processed_messages = set()
try:
    if os.path.exists(PROCESSED_MSGS_FILE):
        with open(PROCESSED_MSGS_FILE, 'r') as f:
            processed_messages = set(json.load(f))
    print("📂 Mensagens processadas carregadas do Drive")
except Exception as e:
    print(f"⚠️ Erro ao carregar mensagens processadas: {str(e)}")

# Carregar estado persistente
try:
    if os.path.exists(PERSISTENCE_FILE):
        with open(PERSISTENCE_FILE, 'r') as f:
            data = json.load(f)
            mirror_channels = data.get('mirror_channels', {})
            forum_topics = defaultdict(lambda: defaultdict(dict), data.get('forum_topics', {}))
        print("📂 Estado anterior carregado do Drive")
except Exception as e:
    print(f"⚠️ Erro ao carregar persistência: {str(e)}")

# Carregar checkpoints (último ID processado por chat)
last_checkpoints = {}
if os.path.exists(LAST_CHECKPOINT_FILE):
    try:
        with open(LAST_CHECKPOINT_FILE, 'r') as f:
            last_checkpoints = json.load(f)
        print("💾 Checkpoints carregados")
    except Exception as e:
        print(f"⚠️ Erro ao carregar checkpoints: {str(e)}")

async def save_persistence():
    with open(PERSISTENCE_FILE, 'w') as f:
        json.dump({
            'mirror_channels': mirror_channels,
            'forum_topics': {
                k: {inner_k: inner_v for inner_k, inner_v in v.items()}
                for k, v in forum_topics.items()
            }
        }, f)
    print("💾 Dados persistidos no Google Drive")

async def save_processed_messages():
    try:
        with open(PROCESSED_MSGS_FILE, 'w') as f:
            json.dump(list(processed_messages), f)
        print("💾 Mensagens processadas salvas no Google Drive")
    except Exception as e:
        print(f"⚠️ Erro ao salvar mensagens processadas: {str(e)}")

def save_checkpoints():
    try:
        with open(LAST_CHECKPOINT_FILE, 'w') as f:
            json.dump(last_checkpoints, f)
        print("💾 Checkpoints salvos")
    except Exception as e:
        print(f"⚠️ Erro ao salvar checkpoints: {str(e)}")

# Configuração interativa
print("⚠️ Execute no Colab: !pip install telethon nest_asyncio\n")
API_ID = "20357896"
API_HASH = "5115fc44c22564711467334938e79a06"
# SESSION_NAME = f"colab-session-{datetime.now().timestamp()}"

client = TelegramClient(
    session=SESSION_NAME,
    api_id=API_ID,
    api_hash=API_HASH,
    sequential_updates=True
)

class BotState:
    def __init__(self):
        self.semaphore = asyncio.Semaphore(3)

bot_state = BotState()

def clean_title(title):
    return ''.join(c for c in title if c.isalnum() or c in (' ', '_')).strip()[:32]

async def check_chat_protected(chat):
    """Verifica se o chat tem restrições de encaminhamento"""
    try:
        full_chat = await client.get_entity(chat)
        if isinstance(full_chat, Channel):
            return full_chat.noforwards or any(
                'forward' in reason.reason.lower()
                for reason in (full_chat.restriction_reason or [])
            )
        return False
    except Exception as e:
        print(f"⚠️ Erro na verificação: {e.__class__.__name__}: {str(e)}")
        return False

async def get_or_create_mirror(source):
    try:
        is_channel = isinstance(source, Channel)
        if str(source.id) in mirror_channels:
            try:
                return await client.get_entity(PeerChannel(int(mirror_channels[str(source.id)])))
            except Exception:
                del mirror_channels[str(source.id)]
        is_forum = getattr(source, 'forum', False) if is_channel else False
        if is_channel:
            result = await client(CreateChannelRequest(
                title=f"{clean_title(source.title)} [MIRROR]",
                about=f"Cópia de {source.id}",
                megagroup=False,
                forum=is_forum
            ))
            mirror = result.chats[0]
            if is_forum and not mirror.forum:
                await client.edit_channel(mirror, forum=True)
        else:
            result = await client(CreateChatRequest(
                title=f"{clean_title(source.title)} [MIRROR]",
                users=[]
            ))
            mirror = result.chats[0]
        mirror_channels[str(source.id)] = str(mirror.id)
        await save_persistence()
        return mirror
    except Exception as e:
        print(f"❌ Erro criando espelho: {e}")
        return None

async def get_topic_name(source_chat, topic_id):
    try:
        async for msg in client.iter_messages(source_chat, ids=topic_id):
            if hasattr(msg.action, 'title') and msg.action.title:
                return msg.action.title
            return msg.text or f"Tópico {topic_id}"
    except Exception as e:
        print(f"⚠️ Erro ao buscar nome do tópico: {e}")
    return f"Tópico {topic_id}"

async def get_or_create_mirror_topic(source_chat, mirror_chat, original_topic_id):
    with lock:
        if original_topic_id in forum_topics[str(source_chat.id)]:
            return forum_topics[str(source_chat.id)][original_topic_id]

    async for msg in client.iter_messages(source_chat, reverse=True, limit=1, reply_to=original_topic_id):
        if not msg:
            print(f"⚠️ Tópico {original_topic_id} não existe no chat de origem.")
            return None

    topic_name = await get_topic_name(source_chat, original_topic_id)
    result = await client(CreateForumTopicRequest(
        channel=mirror_chat,
        title=clean_title(topic_name),
        random_id=int(datetime.now().timestamp())
    ))

    topic_id = None
    for update in result.updates:
        if hasattr(update, 'message') and hasattr(update.message, 'id'):
            topic_id = update.message.id
            break

    if not topic_id:
        raise ValueError("Falha ao criar tópico espelho")

    with lock:
        forum_topics[str(source_chat.id)][original_topic_id] = topic_id
    await save_persistence()
    return topic_id

async def handle_batch(destination, messages, source_chat, is_protected=False):
    total_success = 0
    max_retries = 3
    is_forum = isinstance(destination, Channel) and getattr(destination, 'forum', False)
    is_group_with_topics = isinstance(destination, Chat) and getattr(destination, 'broadcast', False)

    for attempt in range(max_retries):
        try:
            if is_protected:
                raise ForbiddenError(
                    request=client.forward_messages,
                    message="Chat protegido detectado"
                )
            await client.forward_messages(destination, messages, source_chat)
            print(f"⚡ {len(messages)} mensagens encaminhadas")
            return len(messages)
        except (ForbiddenError, ChatWriteForbiddenError, ChatForwardsRestrictedError) as e:
            print(f"⚠ {e.__class__.__name__}: {str(e)} - Copiando individualmente...")
            success = 0
            topics = defaultdict(list)
            general_messages = []
            for msg in messages:
                if (is_forum or is_group_with_topics) and isinstance(msg.reply_to, MessageReplyHeader):
                    original_topic_id = msg.reply_to.reply_to_top_id or msg.reply_to.reply_to_msg_id
                    if original_topic_id:
                        topics[original_topic_id].append(msg)
                        continue
                general_messages.append(msg)
            for topic_id, msgs in topics.items():
                mirror_topic_id = await get_or_create_mirror_topic(source_chat, destination, topic_id)
                if not mirror_topic_id:
                    continue
                for msg in msgs:
                    try:
                        if await copy_message(destination, msg, is_protected_chat=is_protected, topic_id=mirror_topic_id):
                            success += 1
                            await asyncio.sleep(1)
                    except Exception as e:
                        print(f"❌ Erro processando mensagem {msg.id}: {str(e)}")
            for msg in general_messages:
                try:
                    await copy_message(
                        destination,
                        msg,
                        is_protected_chat=is_protected,
                        topic_id=None
                    )
                    success += 1
                    await asyncio.sleep(1)
                except Exception as e:
                    print(f"❌ Erro processando mensagem {msg.id}: {str(e)}")
            total_success = success
            return total_success
        except FloodWaitError as e:
            wait_time = e.seconds
            print(f"⏳ FloodWait: {wait_time}s (tentativa {attempt + 1})")
            await asyncio.sleep(wait_time)
            continue
        except Exception as e:
            print(f"💣 Erro crítico no batch: {str(e)}")
            return total_success
    print("🚨 Máximo de tentativas atingido")
    return total_success

##############################
# NOVAS FUNÇÕES DE DOWNLOAD  #
##############################
async def safe_download_media(message):
    temp_dir = "/content/temp_downloads"
    os.makedirs(temp_dir, exist_ok=True)
    unique_str = f"{message.id}_{hashlib.md5(str(message.media).encode()).hexdigest()[:6]}"
    ext = ".mp4" if "video" in str(message.media).lower() else ".bin"
    final_path = os.path.join(temp_dir, f"media_{unique_str}{ext}")
    print("Iniciando download do arquivo protegido...")
    await client.download_media(message, file=final_path, progress_callback=download_progress)
    if not os.path.exists(final_path) or os.path.getsize(final_path) == 0:
         raise ValueError("Arquivo vazio ou não baixado corretamente.")
    return final_path

def download_progress(current, total):
    percent = (current / total) * 100 if total else 0
    print(f"Progresso do download: {percent:.1f}%")

#################################
# FIM DAS NOVAS FUNÇÕES DE DOWNLOAD #
#################################

async def copy_message(destination, message, is_protected_chat=False, topic_id=None):
    topic_id = topic_id if topic_id is not None else (1 if (getattr(destination, 'forum', False) or (isinstance(destination, Chat) and getattr(destination, 'broadcast', False))) else None)
    try:
        sender = await message.get_sender()
        date_str = message.date.strftime('%d/%m/%Y %H:%M')
        sender_name = sender.title if isinstance(sender, Channel) else getattr(sender, 'first_name', 'Desconhecido')
        media_tag = ""
        file_name = "arquivo"
        file_ext = ""
        unique_id = str(message.id)
        mime_type = ""
        is_video = False
        file_attributes = []
        if message.media:
            if isinstance(message.media, MessageMediaWebPage):
                media_tag = "[LINK] "
                caption = f"{media_tag}**Origem:** {sender_name}\n**Data:** {date_str}"
                if message.text:
                    link_text = message.text.split('\n')[0]
                    caption += f"\n🔗 {link_text}"
                await client.send_message(
                    destination,
                    caption,
                    reply_to=topic_id,
                    link_preview=False
                )
                return True
            elif isinstance(message.media, MessageMediaPhoto):
                media_tag = "[PHOTO] "
                file_ext = ".jpg"
                file_name = f"foto_{unique_id}"
                file_attributes = [DocumentAttributeFilename(file_name=f"{file_name}{file_ext}")]
                mime_type = "image/jpeg"
            elif isinstance(message.media, MessageMediaDocument):
                doc = message.media.document
                mime_type = doc.mime_type
                file_attributes = doc.attributes
                video_attr = next(
                    (attr for attr in doc.attributes
                     if isinstance(attr, DocumentAttributeVideo)),
                    None
                )
                filename_attr = next(
                    (attr for attr in doc.attributes
                     if isinstance(attr, DocumentAttributeFilename)),
                    None
                )
                is_video = bool(video_attr) or ("video" in mime_type)
                if is_video and not video_attr:
                    duration = getattr(doc, 'duration', 0)
                    file_attributes.append(DocumentAttributeVideo(
                        duration=duration,
                        w=1280,
                        h=720,
                        supports_streaming=True
                    ))
                if filename_attr:
                    base_name, original_ext = os.path.splitext(filename_attr.file_name)
                    file_ext = original_ext.lower() or ".mp4"
                    file_name = f"{base_name}_{unique_id}"
                else:
                    file_ext = ".mp4" if is_video else ".bin"
                    file_name = f"video_{unique_id}" if is_video else f"doc_{unique_id}"
                file_ext = file_ext if file_ext.startswith(".") else f".{file_ext}"
                if is_video and file_ext not in [".mp4", ".mkv", ".mov"]:
                    file_ext = ".mp4"
                media_tag = "[VIDEO] " if is_video else "[FILE] "
        caption = f"{media_tag}**Origem:** {sender_name}\n**Data:** {date_str}"
        if message.text:
            caption += f"\n\n{message.text}"
        if message.media and not isinstance(message.media, MessageMediaWebPage):
            full_name = f"{file_name}{file_ext}".replace("/", "_")[:64]
            async with bot_state.semaphore:
                if is_protected_chat:
                    temp_path = None
                    try:
                        temp_path = await safe_download_media(message)
                        await client.send_file(
                            destination,
                            temp_path,
                            caption=caption[:1024],
                            force_document=False,
                            file_name=full_name,
                            supports_streaming=is_video,
                            attributes=file_attributes,
                            reply_to=topic_id
                        )
                    finally:
                        if temp_path and os.path.exists(temp_path):
                            os.remove(temp_path)
                else:
                    await client.send_file(
                        destination,
                        message.media,
                        caption=caption[:1024],
                        force_document=False,
                        file_name=full_name,
                        supports_streaming=is_video,
                        attributes=file_attributes,
                        reply_to=topic_id
                    )
                return True
        if message.sticker:
            await client.send_file(
                destination,
                message.sticker,
                force_document=False,
                reply_to=topic_id
            )
            return True
        if message.poll:
            try:
                await client.send_poll(
                    destination,
                    question=message.poll.poll.question,
                    answers=[a.text for a in message.poll.poll.answers],
                    close_date=message.poll.poll.close_date,
                    public_voters=False,
                    reply_to=topic_id
                )
                return True
            except errors.PollPublicVotersRequiredError:
                await client.send_poll(
                    destination,
                    question=message.poll.poll.question,
                    answers=[a.text for a in message.poll.poll.answers],
                    close_date=message.poll.poll.close_date,
                    public_voters=False,
                    reply_to=topic_id
                )
                return True
        return False
    except FloodWaitError as e:
        wait_time = e.seconds + 5
        print(f"⏳ FloodWait: Esperando {wait_time}s")
        await asyncio.sleep(wait_time)
        return await copy_message(destination, message, is_protected_chat, topic_id)
    except errors.FileReferenceExpiredError as e:
        print(f"🔄 FileReference expirado. Tentando atualizar... (Mensagem {message.id})")
    except errors.MediaInvalidError as e:
        print(f"🚫 Mídia autodestrutiva não suportada (ID {message.id})")
        return False
    except Exception as e:
        print(f"❌ Erro copiando mensagem {message.id}: {str(e)}")
        return False

async def worker():
    while True:
        user_id, event, url = await QUEUE.get()
        user_state = user_states[user_id]
        try:
            if user_state.active_operations >= MAX_CONCURRENT_PER_USER:
                await event.reply("🚫 Limite de operações atingido")
                return
            user_state.active_operations += 1
            await process_request(event, url)
        finally:
            user_state.active_operations -= 1
            QUEUE.task_done()

async def process_request(event, url):
    mirror = None
    try:
        await safe_join_chat(url)
        source = await client.get_entity(url)
        mirror = await get_or_create_mirror(source)
        if not mirror:
            return await event.reply("❌ Falha ao criar o espelho")
        is_protected = await check_chat_protected(source)
        protected_warning = "⚠️ Chat Protegado Detectado! Usando modo de cópia segura.\n" if is_protected else ""
        invite = await client(ExportChatInviteRequest(
            peer=mirror,
            usage_limit=1,
            title=f"Convite único para {event.sender_id}",
            expire_date=int(time.time()) + 3600
        ))
        await event.reply(
            "⚠️ **Atenção!**\n\n"
            f"1. Entre no grupo espelho usando este link: {invite.link}\n"
            "2. Você tem 10 minutos para entrar\n"
            "3. O link expira após 1 uso\n\n"
            "✅ Envie /confirmar após entrar"
        )
        user_joined = False
        start_time = time.time()
        while time.time() - start_time < 600:
            participants = await client.get_participants(mirror)
            if any(p.id == event.sender_id for p in participants):
                user_joined = True
                break
            await asyncio.sleep(15)
        if not user_joined:
            await event.reply("⌛ Tempo esgotado! Excluindo espelho...")
            await client.delete_dialog(mirror)
            return
        await client.edit_admin(
            entity=mirror,
            user=await client.get_entity(event.sender_id),
            is_admin=True,
            change_info=True,
            delete_messages=True,
            ban_users=True,
            invite_users=True
        )
        # --- INÍCIO: Utilização do checkpoint ---
        # Recupera o último ID processado para este chat (se houver)
        source_id_str = str(source.id)
        if source_id_str in last_checkpoints:
            checkpoint_data = last_checkpoints[source_id_str]
            last_msg_checkpoint = checkpoint_data.get("last_msg_id", 0)
            processed_batches = checkpoint_data.get("batches_processed", 0)
        else:
            last_msg_checkpoint = 0
            processed_batches = 0
        # Itera apenas sobre mensagens com id > last_msg_checkpoint
        all_messages = []
        async for msg in client.iter_messages(source, reverse=True, min_id=last_msg_checkpoint+1):
            if not isinstance(msg, MessageService):
                all_messages.append(msg)
        print(f"Foram encontradas {len(all_messages)} novas mensagens após o checkpoint {last_msg_checkpoint}")
        # --- FIM: Utilização do checkpoint ---

        batch_size = 1 if is_protected else BATCH_SIZE
        total = len(all_messages)
        success_count = 0
        batch_count = (total + batch_size - 1) // batch_size
        await event.reply(
            f"{protected_warning}📦 Iniciando cópia de {total} mensagens em {batch_count} batches"
        )
        max_processed = last_msg_checkpoint

        for i in range(0, total, batch_size):
            batch = all_messages[i:i+batch_size]
            success = await handle_batch(
                destination=mirror,
                messages=batch,
                source_chat=source,
                is_protected=is_protected
            )
            success_count += success

            # Atualiza o checkpoint baseado no maior ID do batch processado
            batch_max = max(msg.id for msg in batch)
            if batch_max > max_processed:
                max_processed = batch_max

            processed_batches += 1

            # Persiste o checkpoint para este chat
            last_checkpoints[source_id_str] = {
                "last_msg_id": max_processed,
                "batches_processed": processed_batches
            }
            save_checkpoints()

            progress = f"Batch {i//batch_size + 1}/{batch_count}: {success}/{len(batch)}"
            await event.reply(f"⏳ Progresso: {progress}")
            if i + batch_size < total:
                await asyncio.sleep(FORWARD_DELAY)

        permanent_invite = await client(ExportChatInviteRequest(
            peer=mirror,
            title=f"Link permanente {datetime.now().date()}",
            usage_limit=0,
            expire_date=None
        ))
        with open(LINKS_DB, "a") as f:
            f.write(f"{event.sender_id}|{mirror.id}|{permanent_invite.link}|{datetime.now()}\n")
        try:
            if isinstance(mirror, Channel):
                print(f"🛑 Saindo do canal (ID: {mirror.id})")
                await client(LeaveChannelRequest(channel=mirror))
            else:
                print(f"🛑 Saindo do grupo (ID: {mirror.id})")
                await client(DeleteChatUserRequest(
                    chat_id=mirror.id,
                    user_id=InputUserSelf()
                ))
        except Exception as remove_error:
            print(f"⚠️ Erro ao sair do chat: {str(remove_error)}")
            await event.reply("⚠️ Erro na saída automática! Remova-me manualmente.")
        await event.reply(
            "✅ Cópia concluída!\n"
            f"📊 Estatísticas finais:\n"
            f"• Total: {total} mensagens\n"
            f"• Sucesso: {success_count}\n"
            f"• Falhas: {total - success_count}\n\n"
            f"🔒 ID do espelho: `{mirror.id}`"
        )
    except errors.FloodWaitError as e:
        await event.reply(f"⏳ Erro de flood! Espere {e.seconds} segundos")
    except Exception as e:
        await event.reply(f"💣 Erro crítico: {str(e)}")
        if mirror:
            try:
                await client.delete_dialog(mirror)
            except Exception as delete_error:
                print(f"⚠️ Erro ao excluir espelho: {str(delete_error)}")
    finally:
        if mirror:
            await client.send_message(
                event.sender_id,
                "⚠️ Link permanente gerado e armazenado com segurança!",
                silent=True
            )

@client.on(events.NewMessage)
async def deduplication_handler(event):
    msg_id = f"{event.chat_id}_{event.message.id}_{event.message.date.timestamp()}"
    if msg_id in processed_messages:
        raise events.StopPropagation
    processed_messages.add(msg_id)
    if len(processed_messages) % 10 == 0:
        await save_processed_messages()

@client.on(events.NewMessage(pattern=r'^/copychat\s+(https://t\.me/(c/|joinchat/|)\S+)$'))
async def handle_copy(event):
    user_id = event.sender_id
    url = event.pattern_match.group(1)
    await QUEUE.put((user_id, event, url))
    await event.reply(f"📥 Posição na fila: {QUEUE.qsize()}")

@client.on(events.NewMessage(pattern='^/status$'))
async def handle_status(event):
    status = [
        f"👥 Usuários ativos: {len(user_states)}",
        f"📚 Fila global: {QUEUE.qsize()}",
        f"🔄 Chats espelho: {len(mirror_channels)}",
        f"🗂️ Tópicos criados: {sum(len(v) for v in forum_topics.values())}"
    ]
    await event.reply("\n".join(status))

@client.on(events.NewMessage(pattern='^/topics$'))
async def handle_topics(event):
    try:
        chat = await event.get_chat()
        if str(chat.id) in forum_topics:
            topics_list = "\n".join([f"• {tag} (ID: {tid})" for tag, tid in forum_topics[str(chat.id)].items()])
            await event.reply(f"📚 Tópicos disponíveis:\n{topics_list}")
        else:
            await event.reply("ℹ️ Nenhum tópico criado neste chat")
    except Exception as e:
        await event.reply(f"💣 Erro: {str(e)}")

async def safe_join_chat(chat_link):
    """Entrar em chats com controle de flood usando link direto"""
    try:
        await client(JoinChannelRequest(chat_link))
    except FloodWaitError as e:
        print(f"⏳ Esperando {e.seconds}s por flood")
        await asyncio.sleep(e.seconds)
        await client(JoinChannelRequest(chat_link))
    except ValueError:
        await client(ImportChatInviteRequest(chat_link.split('/')[-1]))

import signal

async def shutdown():
    print("\n🔌 Desconectando...")
    await save_persistence()
    await save_processed_messages()
    save_checkpoints()
    await client.disconnect()

async def main():
    async def periodic_save():
        while True:
            await asyncio.sleep(300)
            await save_processed_messages()
    for _ in range(WORKERS):
        asyncio.create_task(worker())
    asyncio.create_task(periodic_save())
    await colab_auth()
    print("\n🤖 Bot pronto! Use /copychat <link>")
    await client.run_until_disconnected()

async def colab_auth():
    if os.path.exists(SESSION_NAME + '.session'):
        print("⏩ Usando sessão existente do Drive")
        await client.start()
        return
    print("🔐 Iniciando nova autenticação...")
    await client.start(
        phone=lambda: input("📱 Número com código país: "),
        password=lambda: input("🔐 Senha 2FA (se tiver): "),
        code_callback=lambda: input("✉️ Código de verificação: ")
    )
    client.session.save()
    print("\n✅ Autenticado! Sessão salva no Drive")

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main())
    except KeyboardInterrupt:
        loop.run_until_complete(shutdown())


📂 Mensagens processadas carregadas do Drive
📂 Estado anterior carregado do Drive
💾 Checkpoints carregados
⚠️ Execute no Colab: !pip install telethon nest_asyncio

⏩ Usando sessão existente do Drive

🤖 Bot pronto! Use /copychat <link>
💾 Mensagens processadas salvas no Google Drive
💾 Mensagens processadas salvas no Google Drive
Foram encontradas 157 novas mensagens após o checkpoint 4
⚠ ForbiddenError: RPCError 403: Chat protegido detectado (caused by method) - Copiando individualmente...
Iniciando download do arquivo protegido...
Progresso do download: 0.1%
Progresso do download: 0.1%
Progresso do download: 0.2%
Progresso do download: 0.3%
Progresso do download: 0.3%
Progresso do download: 0.4%
Progresso do download: 0.5%
Progresso do download: 0.5%
Progresso do download: 0.6%
Progresso do download: 0.7%
Progresso do download: 0.7%
Progresso do download: 0.8%
Progresso do download: 0.9%
Progresso do download: 0.9%
Progresso do download: 1.0%
Progresso do download: 1.1%
Progresso do down

A saída de streaming foi truncada nas últimas 5000 linhas.
Progresso do download: 10.9%
Progresso do download: 11.0%
Progresso do download: 11.1%
Progresso do download: 11.3%
Progresso do download: 11.4%
Progresso do download: 11.5%
Progresso do download: 11.7%
Progresso do download: 11.8%
Progresso do download: 11.9%
Progresso do download: 12.1%
Progresso do download: 12.2%
Progresso do download: 12.3%
Progresso do download: 12.5%
Progresso do download: 12.6%
Progresso do download: 12.8%
Progresso do download: 12.9%
Progresso do download: 13.0%
Progresso do download: 13.2%
Progresso do download: 13.3%
Progresso do download: 13.4%
Progresso do download: 13.6%
Progresso do download: 13.7%
Progresso do download: 13.8%
Progresso do download: 14.0%
Progresso do download: 14.1%
Progresso do download: 14.2%
Progresso do download: 14.4%
Progresso do download: 14.5%
Progresso do download: 14.7%
Progresso do download: 14.8%
Progresso do download: 14.9%
Progresso do download: 15.1%
Progresso do 